In [ ]:
pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 25.4 MB/s eta 0:00:00


SSIM, PSNR OF ORIGINAL AND WATERMARKED FRAME  AND SSIM, MSE OF THE GAN GENERATED WATERMARK AND THE EXTRACTED WATERMARK WITHOUT ANY DEEPFAKE


In [7]:
import cv2
import numpy as np
import tensorflow as tf
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

def build_generator():
    """ Define a simple CNN generator for watermark prediction """
    inputs = tf.keras.layers.Input(shape=(128, 128, 1))
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = tf.keras.Model(inputs, x, name="Generator")
    return model

generator = build_generator()
weights_path = "/content/generator_epoch_4900.weights.h5"


if os.path.exists(weights_path):
    generator.load_weights(weights_path)
    print("✅ Generator weights loaded successfully!")
else:
    raise FileNotFoundError(f"❌ Weights file not found at {weights_path}")

INPUT_VIDEO_PATH = "/content/input2.mp4"
OUTPUT_VIDEO_PATH = "./output_watermarked2.mp4"
GENERATED_WATERMARK_PATH = "./generated_watermarks2"
EXTRACTED_WATERMARK_PATH = "./extracted_watermarks2"

os.makedirs(GENERATED_WATERMARK_PATH, exist_ok=True)
os.makedirs(EXTRACTED_WATERMARK_PATH, exist_ok=True)

cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

frame_count = 0
psnr_values = []
ssim_values = []
watermark_ssim_values = []
watermark_mse_values = []

alpha = 0.03

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = frame.astype(np.float32) / 255.0

    gray_frame = cv2.cvtColor((frame * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (128, 128)) / 255.0
    input_frame = np.expand_dims(resized_frame, axis=[0, -1])
    watermark = generator.predict(input_frame)[0, :, :, 0]

    watermark_resized = cv2.resize(watermark, (frame_width, frame_height))

    generated_watermark_path = os.path.join(GENERATED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(generated_watermark_path, (watermark_resized * 255).astype(np.uint8))

    watermarked_frame = frame + alpha * np.expand_dims(watermark_resized, axis=-1)
    watermarked_frame = np.clip(watermarked_frame * 255, 0, 255).astype(np.uint8)

    out.write(watermarked_frame)

    original_frame = (frame * 255).astype(np.uint8)
    psnr_value = psnr(original_frame, watermarked_frame, data_range=255)
    ssim_value = ssim(original_frame, watermarked_frame, data_range=255, win_size=3, channel_axis=-1)

    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)

    extracted_watermark = (watermarked_frame.astype(np.float32) - frame * 255) / alpha
    extracted_watermark = np.mean(extracted_watermark, axis=-1)

    extracted_watermark = cv2.GaussianBlur(extracted_watermark, (3, 3), 0)

    extracted_watermark = np.clip(extracted_watermark, 0, 255).astype(np.uint8)

    extracted_watermark_path = os.path.join(EXTRACTED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(extracted_watermark_path, extracted_watermark)

    h, w = watermark_resized.shape
    h_start, h_end = h // 4, 3 * h // 4
    w_start, w_end = w // 4, 3 * w // 4
    cropped_original = watermark_resized[h_start:h_end, w_start:w_end]
    cropped_extracted = extracted_watermark[h_start:h_end, w_start:w_end]

    watermark_ssim = ssim(cropped_original, cropped_extracted.astype(np.float32) / 255, data_range=1.0)
    watermark_mse = mean_squared_error(cropped_original.flatten(), cropped_extracted.flatten() / 255)

    watermark_ssim_values.append(watermark_ssim)
    watermark_mse_values.append(watermark_mse)

    frame_count += 1

cap.release()
out.release()
avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)
avg_watermark_ssim = np.mean(watermark_ssim_values)
avg_watermark_mse = np.mean(watermark_mse_values)

print(f"✅ Watermarked video saved: {OUTPUT_VIDEO_PATH}")
print(f"✅ Generated watermarks saved in: {GENERATED_WATERMARK_PATH}")
print(f"✅ Extracted watermarks saved in: {EXTRACTED_WATERMARK_PATH}")
print(f"📊  PSNR: {avg_psnr:.4f} (More Than 30 is better)")
print(f"📊  SSIM : {avg_ssim:.6f} (Closer to 1 is better)")
print(f"📊 Watermark SSIM: {avg_watermark_ssim:.4f} (Closer to 1 is better)")
print(f"📊 Watermark MSE: {avg_watermark_mse:.6f} (Lower is better)")
print(f"🎯 Model Accuracy (Based on SSIM): {avg_watermark_ssim * 100:.2f}%")
print(f"Number of Frames:{frame_count}")

✅ Generator weights loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━

EXTRACT THE WATERMARK FROM THE DEEPFAKED WATERMARK VIDEO


In [8]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim

ORIGINAL_VIDEO_PATH = "/content/input2.mp4"
WATERMARKED_VIDEO_PATH = "/content/deepfaked2.mp4"
EXTRACTED_WATERMARK_PATH = "./deepfaked_extracted_watermarks2"

os.makedirs(EXTRACTED_WATERMARK_PATH, exist_ok=True)

cap_original = cv2.VideoCapture(ORIGINAL_VIDEO_PATH)
cap_watermarked = cv2.VideoCapture(WATERMARKED_VIDEO_PATH)

alpha = 0.03

frame_count = 0
ssim_values = []

while cap_original.isOpened() and cap_watermarked.isOpened():
    ret_orig, frame_orig = cap_original.read()
    ret_watermarked, frame_watermarked = cap_watermarked.read()

    if not ret_orig or not ret_watermarked:
        break

    frame_orig = frame_orig.astype(np.float32)
    frame_watermarked = frame_watermarked.astype(np.float32)

    extracted_watermark = (frame_watermarked - frame_orig) / alpha

    extracted_watermark = np.mean(extracted_watermark, axis=-1)

    extracted_watermark = cv2.GaussianBlur(extracted_watermark, (3, 3), 0)

    extracted_watermark = np.clip(extracted_watermark, 0, 255).astype(np.uint8)

    extracted_watermark_path = os.path.join(EXTRACTED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(extracted_watermark_path, extracted_watermark)

    frame_count += 1

cap_original.release()
cap_watermarked.release()

print(f"✅ Extracted watermarks saved in: {EXTRACTED_WATERMARK_PATH}")


✅ Extracted watermarks saved in: ./deepfaked_extracted_watermarks2


Calculating the SSIM of GAN Generated Watermark and extracted watermark from deepfaked video

In [9]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

GENERATED_WATERMARK_PATH = "/content/generated_watermarks2"
EXTRACTED_WATERMARK_PATH = "/content/deepfaked_extracted_watermarks2"

generated_files = sorted(os.listdir(GENERATED_WATERMARK_PATH))
extracted_files = sorted(os.listdir(EXTRACTED_WATERMARK_PATH))

assert len(generated_files) == len(extracted_files), "Mismatch in number of frames!"

SSIM_THRESHOLD = 0.7
MSE_THRESHOLD = 0.02

deepfake_frames = 0
total_frames = len(generated_files)
ssim_values = []
mse_values = []

for gen_file, ext_file in zip(generated_files, extracted_files):
    gen_path = os.path.join(GENERATED_WATERMARK_PATH, gen_file)
    ext_path = os.path.join(EXTRACTED_WATERMARK_PATH, ext_file)

    gen_img = cv2.imread(gen_path, cv2.IMREAD_GRAYSCALE) / 255.0
    ext_img = cv2.imread(ext_path, cv2.IMREAD_GRAYSCALE) / 255.0

    ssim_value = ssim(gen_img, ext_img, data_range=1.0)
    mse_value = mean_squared_error(gen_img.flatten(), ext_img.flatten())

    ssim_values.append(ssim_value)
    mse_values.append(mse_value)

    if ssim_value < SSIM_THRESHOLD or mse_value > MSE_THRESHOLD:
        deepfake_frames += 1

avg_ssim = np.mean(ssim_values)
deepfake_percentage = (deepfake_frames / total_frames) * 100
if avg_ssim>=0.6:
    print("video is authentic you are eligible for copyright")
else:
    print(" you can proceed for further steps")



print(f"📊 Average SSIM: {avg_ssim:.4f} (Closer to 1 is better)")
print(f"🔎 Deepfake Suspicious Frames: {deepfake_frames}/{total_frames} ({deepfake_percentage:.2f}%)")

 you can proceed for further steps
📊 Average SSIM: 0.4705 (Closer to 1 is better)
🔎 Deepfake Suspicious Frames: 455/455 (100.00%)


In [11]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.0 MB/s eta 0:00:00


Extracting the watermark stored in the dwt/dct

In [12]:
import cv2
import numpy as np
import pywt
import os
from tqdm import tqdm

def extract_watermark_from_image(watermarked_img_path, alpha=0.05, watermark_shape=(50, 50)):
    wm_img = cv2.imread(watermarked_img_path, cv2.IMREAD_GRAYSCALE)
    if wm_img is None:
        raise ValueError(f"Could not load image: {watermarked_img_path}")
    coeffs = pywt.dwt2(wm_img, 'haar')
    LL, (LH, HL, HH) = coeffs
    dct_LL = cv2.dct(np.float32(LL))
    estimated_original = cv2.medianBlur(dct_LL, 5)
    watermark_diff = (dct_LL - estimated_original) / alpha
    watermark = cv2.resize(watermark_diff, watermark_shape)
    watermark = cv2.normalize(watermark, None, 0, 255, cv2.NORM_MINMAX)

    return watermark.astype(np.uint8)

def batch_extract_watermarks(input_dir, output_dir, alpha=0.05, watermark_shape=(50, 50)):
    """
    Processes all images in a directory to extract watermarks
    """
    os.makedirs(output_dir, exist_ok=True)
    frame_files = sorted([f for f in os.listdir(input_dir) if f.startswith('frame_') and f.endswith('.png')])
    watermark_accumulator = None
    valid_count = 0
    for frame_file in tqdm(frame_files, desc="Extracting watermarks"):
        try:
            frame_path = os.path.join(input_dir, frame_file)
            wm = extract_watermark_from_image(frame_path, alpha, watermark_shape)
            if watermark_accumulator is None:
                watermark_accumulator = np.zeros_like(wm, dtype=np.float32)
            watermark_accumulator += wm.astype(np.float32)
            valid_count += 1
            output_path = os.path.join(output_dir, f"extracted_{frame_file}")
            cv2.imwrite(output_path, wm)

        except Exception as e:
            print(f"Error processing {frame_file}: {str(e)}")
    if valid_count > 0:
        avg_watermark = (watermark_accumulator / valid_count).astype(np.uint8)
        cv2.imwrite(os.path.join(output_dir, "averaged_watermark.png"), avg_watermark)
        print(f"Successfully processed {valid_count} frames. Average watermark saved.")
    else:
        print("No valid frames processed.")

if __name__ == "__main__":
    INPUT_DIR = "/content/generated_watermarks2"  # Folder with your watermarked frames
    OUTPUT_DIR = "./check_generated2"  # Where to save extracted watermarks
    ALPHA = 0.05  # Must match your embedding strength
    WATERMARK_SHAPE = (50, 50)  # Must match your original watermark dimensions

    # Run extraction
    batch_extract_watermarks(INPUT_DIR, OUTPUT_DIR, ALPHA, WATERMARK_SHAPE)

Extracting watermarks: 100%|██████████| 455/455 [00:14<00:00, 31.99it/s]

Successfully processed 455 frames. Average watermark saved.


Extracting the watermark embeded in the deepfaked frame

In [13]:
import cv2
import numpy as np
import pywt
import os
from tqdm import tqdm

def extract_watermark_from_image(watermarked_img_path, alpha=0.05, watermark_shape=(50, 50)):
    wm_img = cv2.imread(watermarked_img_path, cv2.IMREAD_GRAYSCALE)
    if wm_img is None:
        raise ValueError(f"Could not load image: {watermarked_img_path}")
    coeffs = pywt.dwt2(wm_img, 'haar')
    LL, (LH, HL, HH) = coeffs
    dct_LL = cv2.dct(np.float32(LL))
    estimated_original = cv2.medianBlur(dct_LL, 5)
    watermark_diff = (dct_LL - estimated_original) / alpha
    watermark = cv2.resize(watermark_diff, watermark_shape)
    watermark = cv2.normalize(watermark, None, 0, 255, cv2.NORM_MINMAX)

    return watermark.astype(np.uint8)

def batch_extract_watermarks(input_dir, output_dir, alpha=0.05, watermark_shape=(50, 50)):
    """
    Processes all images in a directory to extract watermarks
    """
    os.makedirs(output_dir, exist_ok=True)
    frame_files = sorted([f for f in os.listdir(input_dir) if f.startswith('frame_') and f.endswith('.png')])
    watermark_accumulator = None
    valid_count = 0
    for frame_file in tqdm(frame_files, desc="Extracting watermarks"):
        try:
            frame_path = os.path.join(input_dir, frame_file)
            wm = extract_watermark_from_image(frame_path, alpha, watermark_shape)
            if watermark_accumulator is None:
                watermark_accumulator = np.zeros_like(wm, dtype=np.float32)
            watermark_accumulator += wm.astype(np.float32)
            valid_count += 1
            output_path = os.path.join(output_dir, f"extracted_{frame_file}")
            cv2.imwrite(output_path, wm)

        except Exception as e:
            print(f"Error processing {frame_file}: {str(e)}")
    if valid_count > 0:
        avg_watermark = (watermark_accumulator / valid_count).astype(np.uint8)
        cv2.imwrite(os.path.join(output_dir, "averaged_watermark.png"), avg_watermark)
        print(f"Successfully processed {valid_count} frames. Average watermark saved.")
    else:
        print("No valid frames processed.")

if __name__ == "__main__":
    INPUT_DIR = "/content/deepfaked_extracted_watermarks2"  # Folder with your watermarked frames
    OUTPUT_DIR = "./deepfaked_extracted2"  # Where to save extracted watermarks
    ALPHA = 0.05  # Must match your embedding strength
    WATERMARK_SHAPE = (50, 50)  # Must match your original watermark dimensions

    # Run extraction
    batch_extract_watermarks(INPUT_DIR, OUTPUT_DIR, ALPHA, WATERMARK_SHAPE)

Extracting watermarks: 100%|██████████| 455/455 [00:10<00:00, 43.67it/s]

Successfully processed 455 frames. Average watermark saved.


Calculating the SSIM for the embeded (50*50)sized watermark before and after deepfake

In [14]:
import cv2
import numpy as np
import os
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm

def calculate_ssim(img1, img2):
    """Calculate SSIM between two images"""
    # Ensure images are the same size
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

    # Convert to float32 for calculation
    img1 = img1.astype(np.float32)
    img2 = img2.astype(np.float32)

    # Calculate SSIM (multichannel=False for grayscale)
    return ssim(img1, img2, data_range=255, full=False)

def compare_folders(generated_dir, extracted_dir):
    """Compare SSIM between corresponding images in two folders"""
    # Get sorted list of files
    gen_files = sorted([f for f in os.listdir(generated_dir) if f.endswith('.png')])
    ext_files = sorted([f for f in os.listdir(extracted_dir) if f.endswith('.png')])

    # Verify we have matching files
    if len(gen_files) != len(ext_files):
        print(f"Warning: Folder count mismatch - Generated: {len(gen_files)}, Extracted: {len(ext_files)}")

    ssim_values = []

    for gen_file, ext_file in tqdm(zip(gen_files, ext_files), total=min(len(gen_files), len(ext_files))):
        gen_path = os.path.join(generated_dir, gen_file)
        ext_path = os.path.join(extracted_dir, ext_file)

        # Load images as grayscale
        gen_img = cv2.imread(gen_path, cv2.IMREAD_GRAYSCALE)
        ext_img = cv2.imread(ext_path, cv2.IMREAD_GRAYSCALE)

        if gen_img is None or ext_img is None:
            print(f"Skipping {gen_file} - couldn't load one or both images")
            continue

        # Calculate SSIM
        current_ssim = calculate_ssim(gen_img, ext_img)
        ssim_values.append(current_ssim)

        print(f"{gen_file} vs {ext_file}: SSIM = {current_ssim:.4f}")

    if ssim_values:
        avg_ssim = np.mean(ssim_values)
        print(f"\nAverage SSIM across {len(ssim_values)} frames: {avg_ssim:.4f}")
        return avg_ssim
    else:
        print("No valid image pairs found")
        return 0

if __name__ == "__main__":
    # Configuration
    GENERATED_DIR = "/content/check_generated2"    # Folder with watermarked images
    EXTRACTED_DIR = "/content/deepfaked_extracted2"    # Folder with extracted watermarks

    # Run comparison
    average_ssim = compare_folders(GENERATED_DIR, EXTRACTED_DIR)
    if average_ssim>=0.7:
        print("video is authentic you are eligible for copyright")
    else:
        print("not eligible for copyright")

    # Save results to file
    with open("ssim_results.txt", "w") as f:
        f.write(f"Average SSIM: {average_ssim:.4f}\n")

 25%|██▍       | 113/456 [00:00<00:00, 1127.00it/s]

averaged_watermark.png vs averaged_watermark.png: SSIM = 0.9653
extracted_frame_0000.png vs extracted_frame_0000.png: SSIM = 0.7553
extracted_frame_0001.png vs extracted_frame_0001.png: SSIM = 0.7467
extracted_frame_0002.png vs extracted_frame_0002.png: SSIM = 0.7501
extracted_frame_0003.png vs extracted_frame_0003.png: SSIM = 0.7550
extracted_frame_0004.png vs extracted_frame_0004.png: SSIM = 0.7517
extracted_frame_0005.png vs extracted_frame_0005.png: SSIM = 0.7403
extracted_frame_0006.png vs extracted_frame_0006.png: SSIM = 0.7430
extracted_frame_0007.png vs extracted_frame_0007.png: SSIM = 0.7332
extracted_frame_0008.png vs extracted_frame_0008.png: SSIM = 0.7360
extracted_frame_0009.png vs extracted_frame_0009.png: SSIM = 0.7249
extracted_frame_0010.png vs extracted_frame_0010.png: SSIM = 0.7227
extracted_frame_0011.png vs extracted_frame_0011.png: SSIM = 0.7271
extracted_frame_0012.png vs extracted_frame_0012.png: SSIM = 0.7394
extracted_frame_0013.png vs extracted_frame_0013.png

 74%|███████▍  | 337/456 [00:00<00:00, 1096.57it/s]

extracted_frame_0222.png vs extracted_frame_0222.png: SSIM = 0.7980
extracted_frame_0223.png vs extracted_frame_0223.png: SSIM = 0.8097
extracted_frame_0224.png vs extracted_frame_0224.png: SSIM = 0.8204
extracted_frame_0225.png vs extracted_frame_0225.png: SSIM = 0.8044
extracted_frame_0226.png vs extracted_frame_0226.png: SSIM = 0.8061
extracted_frame_0227.png vs extracted_frame_0227.png: SSIM = 0.7865
extracted_frame_0228.png vs extracted_frame_0228.png: SSIM = 0.7843
extracted_frame_0229.png vs extracted_frame_0229.png: SSIM = 0.7808
extracted_frame_0230.png vs extracted_frame_0230.png: SSIM = 0.7759
extracted_frame_0231.png vs extracted_frame_0231.png: SSIM = 0.7797
extracted_frame_0232.png vs extracted_frame_0232.png: SSIM = 0.7760
extracted_frame_0233.png vs extracted_frame_0233.png: SSIM = 0.7500
extracted_frame_0234.png vs extracted_frame_0234.png: SSIM = 0.7537
extracted_frame_0235.png vs extracted_frame_0235.png: SSIM = 0.7776
extracted_frame_0236.png vs extracted_frame_0236

100%|██████████| 456/456 [00:00<00:00, 1042.01it/s]

extracted_frame_0436.png vs extracted_frame_0436.png: SSIM = 0.8352
extracted_frame_0437.png vs extracted_frame_0437.png: SSIM = 0.8138
extracted_frame_0438.png vs extracted_frame_0438.png: SSIM = 0.8277
extracted_frame_0439.png vs extracted_frame_0439.png: SSIM = 0.8423
extracted_frame_0440.png vs extracted_frame_0440.png: SSIM = 0.8496
extracted_frame_0441.png vs extracted_frame_0441.png: SSIM = 0.8542
extracted_frame_0442.png vs extracted_frame_0442.png: SSIM = 0.8515
extracted_frame_0443.png vs extracted_frame_0443.png: SSIM = 0.8359
extracted_frame_0444.png vs extracted_frame_0444.png: SSIM = 0.8597
extracted_frame_0445.png vs extracted_frame_0445.png: SSIM = 0.8552
extracted_frame_0446.png vs extracted_frame_0446.png: SSIM = 0.8426
extracted_frame_0447.png vs extracted_frame_0447.png: SSIM = 0.8728
extracted_frame_0448.png vs extracted_frame_0448.png: SSIM = 0.8753
extracted_frame_0449.png vs extracted_frame_0449.png: SSIM = 0.8582
extracted_frame_0450.png vs extracted_frame_0450